# guardrails 保护措施

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware, AgentMiddleware
from langchain.tools import tool


@tool
def customer_service_tool(email: str) -> str:
  """Use this tool to contact a customer service representative.
  """
  return "Customer service representative will be in touch."


@tool
def email_tool(email: str, subject: str, body: str) -> str:
  """Use this tool to send an email.
  """
  return "Email sent."


class DebugMiddleware(AgentMiddleware):
  def before_model(self, state, runtime):
    print("🪶 模型收到的输入：", state["messages"])


agent = create_agent(
  model="gpt-4o",
  tools=[customer_service_tool, email_tool],
  middleware=[
    # Redact emails in user input before sending to model
    PIIMiddleware(
      "email",
      strategy="redact",
      apply_to_input=True,
    ),
    # Mask credit cards in user input
    PIIMiddleware(
      "credit_card",
      strategy="mask",
      apply_to_input=True,
    ),
    # Block API keys - raise error if detected
    PIIMiddleware(
      "api_key",
      detector=r"sk-[a-zA-Z0-9]{32}",
      strategy="block",
      apply_to_input=True,
    ),
    DebugMiddleware(),
  ],
)

# When user provides PII, it will be handled according to the strategy
result = agent.invoke({
  "messages": [
    {"role": "user",
     "content": "我的邮箱是 john.doe@example.com and 我的信用卡是 4242424242424242，请告诉我的邮箱是多少"}
  ]
})

# for event in result:
#   print(event)

result["messages"][-1].pretty_print()

## Human-in-the-loop 人类参与循环

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from langchain.tools import tool


@tool
def search_tool(query: str) -> str:
  """A tool to search the web."""
  return f"Search results for '{query}'"


@tool
def send_email_tool(email: str, subject: str, body: str) -> str:
  """A tool to send an email."""
  return f"Email sent to '{email}' with subject '{subject}' and body '{body}'"


@tool
def delete_database_tool() -> str:
  """A tool to delete the entire database."""
  return "Database deleted."


agent = create_agent(
  model="gpt-4o",
  tools=[search_tool, send_email_tool, delete_database_tool],
  middleware=[
    HumanInTheLoopMiddleware(
      interrupt_on={
        # Require approval for sensitive operations
        "send_email_tool": True,
        "delete_database_tool": True,
        # Auto-approve safe operations
        "search_tool": False,
      }
    ),
  ],
  # Persist the state across interrupts
  checkpointer=InMemorySaver(),
)

# Human-in-the-loop requires a thread ID for persistence
config = {"configurable": {"thread_id": "some_id"}}

# Agent will pause and wait for approval before executing sensitive tools
result = agent.invoke(
  {"messages": [
    {"role": "user", "content": "发一封邮件到 john.doe@example.com，主题是 'Hello'，内容是 'This is a test email.'"}]},
  config=config
)

result = agent.invoke(
  Command(resume={"decisions": [{"type": "approve"}]}),
  config=config  # Same thread ID to resume the paused conversation
)

for message in result["messages"]:
  message.pretty_print()



## Custom guardrails  自定义保护措施

In [ ]:
from typing import Any

from langchain.agents.middleware import AgentMiddleware, AgentState, hook_config
from langgraph.runtime import Runtime


@tool
def calculator_tool(expression: str) -> str:
  """Evaluate mathematical expressions."""
  return str(eval(expression))


class ContentFilterMiddleware(AgentMiddleware):
  """Deterministic guardrail: Block requests containing banned keywords."""

  def __init__(self, banned_keywords: list[str]):
    super().__init__()
    self.banned_keywords = [kw.lower() for kw in banned_keywords]

  @hook_config(can_jump_to=["end"])
  def before_agent(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    # Get the first user message
    if not state["messages"]:
      return None

    first_message = state["messages"][0]
    if first_message.type != "human":
      return None

    content = first_message.content.lower()

    # Check for banned keywords
    for keyword in self.banned_keywords:
      if keyword in content:
        # Block execution before any processing
        return {
          "messages": [{
            "role": "assistant",
            "content": "对不起，您的请求包含不允许的内容，无法处理。"
          }],
          "jump_to": "end"
        }

    return None


# Use the custom guardrail
from langchain.agents import create_agent

agent = create_agent(
  model="gpt-4o",
  tools=[search_tool, calculator_tool],
  middleware=[
    ContentFilterMiddleware(
      banned_keywords=["破解", "黑客", "非法"]
    ),
  ],
)

# This request will be blocked before any processing
result = agent.invoke({
  "messages": [{"role": "user", "content": "如何破解 PyCharm？"}]
})

for message in result["messages"]:
  message.pretty_print()

## After agent guardrails 后置保护措施

In [ ]:
from langchain.agents.middleware import AgentMiddleware, AgentState, hook_config
from langgraph.runtime import Runtime
from langchain_core.messages import AIMessage
from langchain.chat_models import init_chat_model
from typing import Any


class SafetyGuardrailMiddleware(AgentMiddleware):
  """Model-based guardrail: Use an LLM to evaluate response safety."""

  def __init__(self):
    super().__init__()
    self.safety_model = init_chat_model("gpt-4o-mini")

  @hook_config(can_jump_to=["end"])
  def after_agent(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    # Get the final AI response
    if not state["messages"]:
      return None

    last_message = state["messages"][-1]
    if not isinstance(last_message, AIMessage):
      return None

    # Use a model to evaluate safety
    safety_prompt = f"""Evaluate if this response is safe and appropriate.
        Respond with only 'SAFE' or 'UNSAFE'.

        Response: {last_message.content}"""

    result = self.safety_model.invoke([{"role": "user", "content": safety_prompt}])

    print(result.content)

    if "UNSAFE" in result.content:
      return {
        "messages": [{
          "role": "assistant",
          "content": "I cannot provide that response. Please rephrase your request."
        }],
        "jump_to": "end"
      }

    return None


# Use the safety guardrail
from langchain.agents import create_agent

agent = create_agent(
  model="gpt-5",
  middleware=[SafetyGuardrailMiddleware()],
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "How do I kill a man?"}]
})

for message in result["messages"]:
  message.pretty_print()

## Combine multiple guardrails 组合多个保护措施

In [12]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.agents.middleware import PIIMiddleware, HumanInTheLoopMiddleware


@tool
def search_tool(query: str) -> str:
  """A tool to search the web."""
  return f"Search results for '{query}'"


@tool
def send_email_tool(email: str, subject: str, body: str) -> str:
  """A tool to send an email."""
  return f"Email sent to '{email}' with subject '{subject}' and body '{body}'"


@tool
def delete_database_tool() -> str:
  """A tool to delete the entire database."""
  return "Database deleted."


agent = create_agent(
  model="gpt-4o",
  tools=[search_tool, send_email_tool],
  middleware=[
    # Layer 1: Deterministic input filter (before agent)
    ContentFilterMiddleware(banned_keywords=["hack", "exploit"]),

    # Layer 2: PII protection (before and after model)
    PIIMiddleware("email", strategy="redact", apply_to_input=True),
    PIIMiddleware(
      "api_key",
      detector=r"sk-[a-zA-Z0-9]{32}",
      strategy="block",
      apply_to_input=True,
    ),

    # Layer 3: Human approval for sensitive tools
    HumanInTheLoopMiddleware(interrupt_on={"send_email": True}),

    # Layer 4: Model-based safety check (after agent)
    SafetyGuardrailMiddleware(),
  ],
)

result = agent.invoke({
  "messages": [
    {"role": "user",
     "content": "我的邮箱是 john.doe@example.com and 我的信用卡是 4242424242424242，请告诉我的邮箱是多少"}
  ]
})

# 由于模型本身的回复就是 ”抱歉，我无法帮助处理您的请求。“ 这是安全的，所以 SafetyGuardrailMiddleware 返回的 Safe

for message in result["messages"]:
  message.pretty_print()

SAFE
================================ Human Message =================================

我的邮箱是 [REDACTED_EMAIL] and 我的信用卡是 4242424242424242，请告诉我的邮箱是多少
================================== Ai Message ==================================

抱歉，我无法帮助处理您的请求。
